# Project 2: GANs
#### Malachi Eberly

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"

### 1. Dataset Preperation

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 64

trainset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2
)

testset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=2
)

### 2. GAN Architecture

In [ ]:
class Generator(nn.Module):
    pass

class Discriminator(nn.Module):
    pass

generator = Generator().to(device)
discriminator = Discriminator().to(device)

### 3. Loss and Optimizer

In [ ]:
g_criterion = nn.CrossEntropyLoss()
g_optimizer = optim.Adam(generator.parameters(), lr=0.01)

d_criterion = nn.CrossEntropyLoss()
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.01)

### 4. Training Loop

In [ ]:
num_epoches = 50

for epoch in range(num_epoches):
    break

### 5. Visualization